In [10]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
from scipy.stats import trim_mean
from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference import settings
from higgs_inference.various.utils import format_number

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
result_dir = '../results/'

# TablePrinter class

In [7]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []
        self.content_in_last_block = False

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):
        
        self.content_in_last_block = False
        
        # Skip if block is empty
        if len(self.block_entries) == 0:
            return

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Skip entirely empty lines
            try:
                if not np.any(np.isfinite(line[2:])):
                    continue
            except TypeError:
                print(line)
                continue
                
            self.content_in_last_block = True
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], latex_math_mode=True, emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], latex_math_mode=True, emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        if self.content_in_last_block:
            self.table += self.indent + self.midrule + self.end_line
            self.content_in_last_block = False
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        if len(self.block_entries) > 0:
            for entry in self.block_entries:
                if entry[0] == col1:
                    line = ['', col2]
                    break
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except IOError:
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
            except ValueError:
                #print('File', filename, 'in folder', folder, 'leads to ValueError')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        if np.any(np.isfinite(line[2:])):
            self.block_entries.append(line)
            self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [22]:
def expected_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/mse_logr_' + filename + '.npy')
    return np.sum(settings.theta_prior * mse_log_r)
                       
def expected_trimmed_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/trimmed_mse_logr_' + filename + '.npy')
    return np.sum(settings.theta_prior * mse_log_r)

In [23]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return (mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return (mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return (trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return (trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return (np.median(errors))

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return (mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return (mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return (trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return (trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.median(errors)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    try:
        return (mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return (mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return (mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return (mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return (mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return (trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return (mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return (trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    return (np.var(expected_log_r_truth - expected_log_r_estimated))

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return (np.var(expected_log_r_truth - expected_log_r_estimated))

# Paper summary table

In [24]:
table = TablePrinter([expected_trimmed_mse_log_r], precisions=[4])

table.add('Histogram', r'$p_{T,j1}, \Delta \phi_{jj}$', 'histo_2d_new', 'histo')
table.add('Histogram', r'$p_{T,j1}$', 'histo_ptj_new', 'histo')
table.add('Histogram', r'$\Delta \phi_{jj}$', 'histo_deltaphi_new', 'histo')
table.add('AFC', r'$p_{T,j1}, \Delta \phi_{jj}$', 'afc_2d_epsilon_0.10', 'afc')
table.add('AFC', r'$p_{T,j1}, m_{Z2}, m_{jj}, \Delta \eta_{jj}, \Delta \phi_{jj}$', 'afc_5d_epsilon_0.50', 'afc')

table.new_block()
table.add('carl (PbP)', 'PbP', 'carl_calibrated', 'point_by_point')
table.add('carl (parameterized)', 'Baseline', 'carl_calibrated_shallow_new', 'parameterized')
table.add('carl (parameterized)', r'Random $\theta$', 'carl_calibrated_random_shallow', 'parameterized')
table.add('carl (morphing-aware)', 'Baseline', 'carl_calibrated_aware', 'parameterized')
table.add('carl (morphing-aware)', r'Random $\theta$', 'carl_calibrated_aware_random', 'parameterized')
table.add('carl (morphing-aware)', r'Morphing basis', 'carl_calibrated_aware_basis', 'parameterized')

table.new_block()
table.add('Local score regression', 'Score density estimation', 'scoreregression_score', 'score_regression')
table.add('Local score regression', r'$t \cdot \Delta \theta$ density estimation', 'scoreregression_scoretheta', 'score_regression')

table.new_block()
table.add('Ratio regression (PbP)', 'PbP', 'regression_calibrated', 'point_by_point')
table.add('Ratio regression (parameterized)', 'Baseline', 'regression_calibrated', 'parameterized')
table.add('Ratio regression (parameterized)', r'Random $\theta$', 'regression_calibrated_random', 'parameterized')
table.add('Ratio regression (morphing-aware)', 'Baseline', 'regression_calibrated_aware', 'parameterized')
table.add('Ratio regression (morphing-aware)', r'Random $\theta$', 'regression_calibrated_aware_random', 'parameterized')
table.add('Ratio regression (morphing-aware)', r'Morphing basis', 'regression_calibrated_aware_basis', 'parameterized')

table.new_block()
table.add('carl + score regression (parameterized)', 'Baseline', 'combined_calibrated_deep', 'parameterized')
table.add('carl + score regression (parameterized)', r'Random $\theta$', 'combined_calibrated_random_deep', 'parameterized')
table.add('carl + score regression (morphing-aware)', 'Baseline', 'combined_calibrated_aware', 'parameterized')
table.add('carl + score regression (morphing-aware)', r'Random $\theta$', 'combined_calibrated_aware_random', 'parameterized')
table.add('carl + score regression (morphing-aware)', r'Morphing basis', 'combined_calibrated_aware_basis', 'parameterized')

table.new_block()
table.add('ratio + score regression (parameterized)', 'Baseline', 'combinedregression_calibrated_deep', 'parameterized')
table.add('ratio + score regression (parameterized)', r'Random $\theta$', 'combinedregression_calibrated_random_deep', 'parameterized')
table.add('ratio + score regression (morphing-aware)', 'Baseline', 'combinedregression_calibrated_aware', 'parameterized')
table.add('ratio + score regression (morphing-aware)', r'Random $\theta$', 'combinedregression_calibrated_aware_random', 'parameterized')
table.add('ratio + score regression (morphing-aware)', r'Morphing basis', 'combinedregression_calibrated_aware_basis', 'parameterized')

print(table.print())


   Histogram & $p_{T,j1}, \Delta \phi_{jj}$ & $0.0101$\\
    & $p_{T,j1}$ & $0.0249$\\
    & $\Delta \phi_{jj}$ & $0.0452$\\
   AFC & $p_{T,j1}, \Delta \phi_{jj}$ & $0.0103$\\
    & $p_{T,j1}, m_{Z2}, m_{jj}, \Delta \eta_{jj}, \Delta \phi_{jj}$ & $\mathbf{0.0101}$\\
   \midrule
   carl (PbP) & PbP & $0.0111$\\
   carl (parameterized) & Baseline & $\mathbf{0.0029}$\\
    & Random $\theta$ & $0.0031$\\
   carl (morphing-aware) & Baseline & $0.0282$\\
    & Random $\theta$ & $0.0289$\\
   \midrule
   Local score regression & Score density estimation & $\mathbf{0.0006}$\\
    & $t \cdot \Delta \theta$ density estimation & $0.0010$\\
   \midrule
   Ratio regression (PbP) & PbP & $0.0022$\\
   Ratio regression (parameterized) & Baseline & $\mathbf{0.0013}$\\
    & Random $\theta$ & $0.0015$\\
   Ratio regression (morphing-aware) & Baseline & $0.0088$\\
    & Random $\theta$ & $0.0089$\\
   \midrule
   carl + score regression (parameterized) & Baseline & $\mathbf{0.0002}$\\
    & Random $\the

# Main tables

In [32]:
labels = ['Histogram', # 0
           r'AFC',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl (aware, raw)',
          'carl (aware, cal.)',
          
          'SM score regr.\ + density est.', # 8
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'carl + score (aware, raw)',
          'carl + score (aware, cal.)',
          
          'Ratio regression (PbP, raw)', # 13
          'Ratio regression (PbP, cal.)',
          'Ratio regression (param., raw)',
          'Ratio regression (param., cal.)',
          'Ratio regression (aware, raw)',
          'Ratio regression (aware, cal.)',
          
          r'Ratio + score regr.\ (param., raw)', # 19
          r'Ratio + score regr.\ (param., cal.)',
          r'Ratio + score regr.\ (aware, raw)',
          r'Ratio + score regr.\ (aware, cal.)']

folders = (['histo'] + ['afc'] + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression'] + ['parameterized'] * 4
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['parameterized'] * 4)

filenames = ['histo',
             'afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'carl_aware',
             'carl_calibrated_aware',
             
             'scoreregression',
             'combined',
             'combined_calibrated',
             'combined_aware',
             'combined_calibrated_aware',
             
             'regression',
             'regression_calibrated',
             'regression',
             'regression_calibrated',
             'regression_aware',
             'regression_calibrated_aware',
             
             'combinedregression',
             'combinedregression_calibrated',
             'combinedregression_aware',
             'combinedregression_calibrated_aware']

print(len(labels), len(filenames), len(folders))


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    #table = TablePrinter([trimmed_mse_log_r_nottrained, mse_expected_log_r],
    #                      precisions=[3,2])
    table = TablePrinter([expected_trimmed_mse_log_r],
                          precisions=[5])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):
        
        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            table.add(label, 'PbP', filename, 'point_by_point')
            table.add(label, 'PbP, shallow', filename + '_shallow', 'point_by_point')
            table.add(label, 'PbP, deep', filename + '_deep', 'point_by_point')

        elif folder == 'histo':
            table.add(label, r'$p_{T,j1}$', filename + '_ptj', 'histo')
            table.add(label, r'$\Delta \phi_{jj}$', filename + '_deltaphi', 'histo')
            table.add(label, '2d', filename + '_2d', 'histo')

        elif folder == 'afc':
            table.add(label, '2d, $\epsilon = 1$', filename + '_2d_epsilon_1.00', 'afc')
            table.add(label, '2d, $\epsilon = 0.5$', filename + '_2d_epsilon_0.50', 'afc')
            table.add(label, '2d, $\epsilon = 0.2$', filename + '_2d_epsilon_0.20', 'afc')
            table.add(label, '2d, $\epsilon = 0.1$', filename + '_2d_epsilon_0.10', 'afc')
            table.add(label, '2d, $\epsilon = 0.05$', filename + '_2d_epsilon_0.05', 'afc')
            table.add(label, '2d, $\epsilon = 0.02$', filename + '_2d_epsilon_0.02', 'afc')
            table.add(label, '2d, $\epsilon = 0.01$', filename + '_2d_epsilon_0.01', 'afc')

            table.add(label, '5d, $\epsilon = 1$', filename + '_5d_epsilon_1.00', 'afc')
            table.add(label, '5d, $\epsilon = 0.5$', filename + '_5d_epsilon_0.50', 'afc')
            table.add(label, '5d, $\epsilon = 0.2$', filename + '_5d_epsilon_0.20', 'afc')
            table.add(label, '5d, $\epsilon = 0.1$', filename + '_5d_epsilon_0.10', 'afc')
            table.add(label, '5d, $\epsilon = 0.05$', filename + '_5d_epsilon_0.05', 'afc')
            table.add(label, '5d, $\epsilon = 0.02$', filename + '_5d_epsilon_0.02', 'afc')
            table.add(label, '5d, $\epsilon = 0.01$', filename + '_5d_epsilon_0.01', 'afc')

        elif folder == 'score_regression':
            table.add(label, r'Fixed density est. on $t$', filename + '_score', 'score_regression')
            table.add(label, r'Dyn. dens. est. on $t$', filename + '_rotatedscore', 'score_regression')
            table.add(label, r'Dens. est. on $t \cdot \theta$', filename + '_scoretheta', 'score_regression')

        else:
            table.add(label, r'Baseline', filename)
            table.add(label, r'Baseline, shallow', filename + '_shallow')
            table.add(label, r'Baseline, deep', filename + '_deep')
            
            table.add(label, r'Baseline, $\alpha = 0.01$', filename + '_alpha_0.010')
            table.add(label, r'Baseline, $\alpha = 0.02$', filename + '_alpha_0.020')
            table.add(label, r'Baseline, $\alpha = 0.05$', filename + '_alpha_0.050')
            table.add(label, r'Baseline, $\alpha = 0.1$', filename + '_alpha_0.10')
            table.add(label, r'Baseline, $\alpha = 0.2$', filename + '_alpha_0.20')
            table.add(label, r'Baseline, $\alpha = 0.5$', filename + '_alpha_0.50')
            table.add(label, r'Baseline, $\alpha = 1$', filename + '_alpha_1.0')
            table.add(label, r'Baseline, $\alpha = 2$', filename + '_alpha_2.0')
            table.add(label, r'Baseline, $\alpha = 5$', filename + '_alpha_5.0')
            table.add(label, r'Baseline, $\alpha = 10$', filename + '_alpha_10')
            table.add(label, r'Baseline, $\alpha = 20$', filename + '_alpha_20')
            table.add(label, r'Baseline, $\alpha = 50$', filename + '_alpha_50')
            table.add(label, r'Baseline, $\alpha = 100$', filename + '_alpha_100')
            table.add(label, r'Baseline, $\alpha = 200$', filename + '_alpha_200')
            
            table.add(label, r'Baseline, large batches', filename + '_largebatch')
            table.add(label, r'Baseline, small batches', filename + '_smallbatch')
            table.add(label, r'Baseline, const.\ LR', filename + '_constantlr')
            table.add(label, r'Baseline, const.\ LR, large batches', filename + '_constantlr_largebatch')
            table.add(label, r'Baseline, const.\ LR, small batches', filename + '_constantlr_largebatch')
            table.add(label, r'Baseline, small LR', filename + '_slowlearning')
            table.add(label, r'Baseline, small LR, large batches', filename + '_slowlearning_largebatch')
            table.add(label, r'Baseline, small LR, small batches', filename + '_slowlearning_smallbatch')
            table.add(label, r'Baseline, small const.\ LR', filename + '_slowlearning_constantlr')
            table.add(label, r'Baseline, small const.\ LR, large batches', filename + '_slowlearning_constantlr_largebatch')
            table.add(label, r'Baseline, small const.\ LR, small batches', filename + '_slowlearning_constantlr_smallbatch')
            table.add(label, r'Baseline, large LR', filename + '_fastlearning')
            table.add(label, r'Baseline, large LR, large batches', filename + '_fastlearning_largebatch')
            table.add(label, r'Baseline, large LR, small batches', filename + '_fastlearning_smallbatch')
            table.add(label, r'Baseline, large const.\ LR', filename + '_fastlearning_constantlr')
            table.add(label, r'Baseline, large const.\ LR, large batches', filename + '_fastlearning_constantlr_largebatch')
            table.add(label, r'Baseline, large const.\ LR, small batches', filename + '_fastlearning_constantlr_smallbatch')
            
            table.add(label, r'Random $\boldtheta$', filename + '_random')
            table.add(label, r'Random $\boldtheta$, shallow', filename + '_random_shallow')
            table.add(label, r'Random $\boldtheta$, deep', filename + '_random_deep')
            
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.01$', filename + '_random_alpha_0.010')
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.02$', filename + '_random_alpha_0.020')
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.05$', filename + '_random_alpha_0.050')
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.1$', filename + '_random_alpha_0.10')
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.2$', filename + '_random_alpha_0.20')
            table.add(label, r'Random $\boldtheta$, $\alpha = 0.5$', filename + '_random_alpha_0.50')
            table.add(label, r'Random $\boldtheta$, $\alpha = 1$', filename + '_random_alpha_1.0')
            table.add(label, r'Random $\boldtheta$, $\alpha = 2$', filename + '_random_alpha_2.0')
            table.add(label, r'Random $\boldtheta$, $\alpha = 5$', filename + '_random_alpha_5.0')
            table.add(label, r'Random $\boldtheta$, $\alpha = 10$', filename + '_random_alpha_10')
            table.add(label, r'Random $\boldtheta$, $\alpha = 20$', filename + '_random_alpha_20')
            table.add(label, r'Random $\boldtheta$, $\alpha = 50$', filename + '_random_alpha_50')
            table.add(label, r'Random $\boldtheta$, $\alpha = 100$', filename + '_random_alpha_100')
            table.add(label, r'Random $\boldtheta$, $\alpha = 200$', filename + '_random_alpha_200')
            
            table.add(label, r'Morphing basis', filename + '_basis')
            table.add(label, r'Morphing basis, shallow', filename + '_basis_shallow')
            table.add(label, r'Morphing basis, deep', filename + '_basis_deep')

    print(table.print())

23 23 23


In [33]:
show_main_table(0,8)

   Histogram & $p_{T,j1}$ & $0.08074$\\
    & $\Delta \phi_{jj}$ & $0.09902$\\
    & 2d & $\mathbf{0.02936}$\\
   \midrule
   AFC & 2d, $\epsilon = 1$ & $0.07573$\\
    & 2d, $\epsilon = 0.5$ & $0.05361$\\
    & 2d, $\epsilon = 0.2$ & $0.03181$\\
    & 2d, $\epsilon = 0.1$ & $\mathbf{0.02858}$\\
    & 2d, $\epsilon = 0.05$ & $0.03308$\\
    & 2d, $\epsilon = 0.02$ & $0.06832$\\
    & 2d, $\epsilon = 0.01$ & $0.21004$\\
    & 5d, $\epsilon = 1$ & $0.06738$\\
    & 5d, $\epsilon = 0.5$ & $0.03761$\\
    & 5d, $\epsilon = 0.2$ & $0.05649$\\
    & 5d, $\epsilon = 0.1$ & $0.88544$\\
    & 5d, $\epsilon = 0.05$ & $13.20712$\\
    & 5d, $\epsilon = 0.02$ & $240.53337$\\
    & 5d, $\epsilon = 0.01$ & $16891.74541$\\
   \midrule
   carl (PbP, raw) & PbP & $\mathbf{0.02276}$\\
   \midrule
   carl (PbP, cal.) & PbP & $\mathbf{0.01533}$\\
   \midrule
   carl (param., raw) & Baseline & $0.00689$\\
    & Baseline, shallow & $\mathbf{0.00597}$\\
    & Baseline, deep & $0.00670$\\
    & Random $\boldt

In [34]:
show_main_table(8,13)

   SM score regr.\ + density est. & Fixed density est. on $t$ & $\mathbf{0.00194}$\\
    & Dyn. dens. est. on $t$ & $0.00205$\\
    & Dens. est. on $t \cdot \theta$ & $0.00378$\\
   \midrule
   carl + score (param., raw) & Baseline & $0.00054$\\
    & Baseline, shallow & $0.00067$\\
    & Baseline, deep & $0.00068$\\
    & Baseline, $\alpha = 0.1$ & $0.00143$\\
    & Baseline, $\alpha = 0.2$ & $0.00110$\\
    & Baseline, $\alpha = 0.5$ & $0.00083$\\
    & Baseline, $\alpha = 2$ & $0.00057$\\
    & Baseline, $\alpha = 5$ & $0.00053$\\
    & Baseline, $\alpha = 10$ & $0.00096$\\
    & Baseline, $\alpha = 20$ & $0.00071$\\
    & Baseline, $\alpha = 50$ & $0.00084$\\
    & Baseline, $\alpha = 100$ & $0.00107$\\
    & Random $\boldtheta$ & $0.00076$\\
    & Random $\boldtheta$, shallow & $0.00085$\\
    & Random $\boldtheta$, deep & $0.00052$\\
    & Random $\boldtheta$, $\alpha = 0.1$ & $0.00129$\\
    & Random $\boldtheta$, $\alpha = 0.2$ & $0.00114$\\
    & Random $\boldtheta$, $\alpha =

In [36]:
show_main_table(13,19)

   Ratio regression (PbP, raw) & PbP & $\mathbf{0.00480}$\\
   \midrule
   Ratio regression (PbP, cal.) & PbP & $\mathbf{0.00463}$\\
   \midrule
   Ratio regression (param., raw) & Baseline & $0.00297$\\
    & Baseline, shallow & $0.00521$\\
    & Baseline, deep & $0.00312$\\
    & Random $\boldtheta$ & $\mathbf{0.00296}$\\
    & Random $\boldtheta$, shallow & $0.00512$\\
    & Random $\boldtheta$, deep & $0.00323$\\
   \midrule
   Ratio regression (param., cal.) & Baseline & $0.00277$\\
    & Baseline, shallow & $0.00467$\\
    & Baseline, deep & $0.00286$\\
    & Random $\boldtheta$ & $\mathbf{0.00276}$\\
    & Random $\boldtheta$, shallow & $0.00456$\\
    & Random $\boldtheta$, deep & $0.00302$\\
   \midrule
   Ratio regression (aware, raw) & Random $\boldtheta$ & $\mathbf{0.07205}$\\
   \midrule
   Ratio regression (aware, cal.) & Random $\boldtheta$ & $\mathbf{0.02501}$\\



In [15]:
show_main_table(19,None)

   Ratio + score regr.\ (param., raw) & Baseline & $0.00070$\\
    & Baseline, shallow & $0.00088$\\
    & Baseline, deep & $0.00105$\\
    & Baseline, $\alpha = 5$ & $0.00090$\\
    & Baseline, $\alpha = 10$ & $0.00075$\\
    & Baseline, $\alpha = 50$ & $0.00068$\\
    & Baseline, $\alpha = 100$ & $0.00071$\\
    & Baseline, $\alpha = 200$ & $0.00061$\\
    & Random $\boldtheta$ & $0.00065$\\
    & Random $\boldtheta$, shallow & $0.00129$\\
    & Random $\boldtheta$, deep & $0.00073$\\
    & Random $\boldtheta$, $\alpha = 5$ & $0.00109$\\
    & Random $\boldtheta$, $\alpha = 10$ & $0.00087$\\
    & Random $\boldtheta$, $\alpha = 50$ & $0.00068$\\
    & Random $\boldtheta$, $\alpha = 100$ & $\mathbf{0.00061}$\\
    & Random $\boldtheta$, $\alpha = 200$ & $0.00064$\\
   \midrule
   Ratio + score regr.\ (param., cal.) & Baseline & $0.00070$\\
    & Baseline, shallow & $0.00086$\\
    & Baseline, deep & $0.00066$\\
    & Baseline, $\alpha = 5$ & $0.00089$\\
    & Baseline, $\alpha = 10$ &

# Score accuracy table

In [28]:
labels = ['carl (param.)',
          'carl (aware)',
          'carl + score regr.\ (param.)',
          'carl + score regr.\ (aware)',
          'Ratio regression (param.)',
          'Ratio regression (aware)',
          'Ratio + score regr.\ (param.)',
          'Ratio + score regr.\ (aware)']

filenames = ['carl',
             'carl_aware',
             'combined',
             'combined_aware',
             'regression',
             'regression_aware',
             'combinedregression',
             'combinedregression_aware']

table = TablePrinter([trimmed_mse_score_nottrained],
                      precisions=[4])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table.new_block()
        
    table.add(label, r'Baseline', filename)
    table.add(label, r'Baseline, shallow', filename + '_shallow')
    table.add(label, r'Baseline, deep', filename + '_deep')

    table.add(label, r'Baseline, $\alpha = 0.01$', filename + '_alpha_0.010')
    table.add(label, r'Baseline, $\alpha = 0.02$', filename + '_alpha_0.020')
    table.add(label, r'Baseline, $\alpha = 0.05$', filename + '_alpha_0.050')
    table.add(label, r'Baseline, $\alpha = 0.1$', filename + '_alpha_0.10')
    table.add(label, r'Baseline, $\alpha = 0.2$', filename + '_alpha_0.20')
    table.add(label, r'Baseline, $\alpha = 0.5$', filename + '_alpha_0.50')
    table.add(label, r'Baseline, $\alpha = 1$', filename + '_alpha_1.0')
    table.add(label, r'Baseline, $\alpha = 2$', filename + '_alpha_2.0')
    table.add(label, r'Baseline, $\alpha = 5$', filename + '_alpha_5.0')
    table.add(label, r'Baseline, $\alpha = 10$', filename + '_alpha_10')
    table.add(label, r'Baseline, $\alpha = 20$', filename + '_alpha_20')
    table.add(label, r'Baseline, $\alpha = 50$', filename + '_alpha_50')
    table.add(label, r'Baseline, $\alpha = 100$', filename + '_alpha_100')

    table.add(label, r'Baseline, large batches', filename + '_largebatch')
    table.add(label, r'Baseline, small batches', filename + '_smallbatch')
    table.add(label, r'Baseline, const.\ LR', filename + '_constantlr')
    table.add(label, r'Baseline, const.\ LR, large batches', filename + '_constantlr_largebatch')
    table.add(label, r'Baseline, const.\ LR, small batches', filename + '_constantlr_largebatch')
    table.add(label, r'Baseline, small LR', filename + '_slowlearning')
    table.add(label, r'Baseline, small LR, large batches', filename + '_slowlearning_largebatch')
    table.add(label, r'Baseline, small LR, small batches', filename + '_slowlearning_smallbatch')
    table.add(label, r'Baseline, small const.\ LR', filename + '_slowlearning_constantlr')
    table.add(label, r'Baseline, small const.\ LR, large batches', filename + '_slowlearning_constantlr_largebatch')
    table.add(label, r'Baseline, small const.\ LR, small batches', filename + '_slowlearning_constantlr_smallbatch')
    table.add(label, r'Baseline, large LR', filename + '_fastlearning')
    table.add(label, r'Baseline, large LR, large batches', filename + '_fastlearning_largebatch')
    table.add(label, r'Baseline, large LR, small batches', filename + '_fastlearning_smallbatch')
    table.add(label, r'Baseline, large const.\ LR', filename + '_fastlearning_constantlr')
    table.add(label, r'Baseline, large const.\ LR, large batches', filename + '_fastlearning_constantlr_largebatch')
    table.add(label, r'Baseline, large const.\ LR, small batches', filename + '_fastlearning_constantlr_smallbatch')

    table.add(label, r'Random $\boldtheta$', filename + '_random')
    table.add(label, r'Random $\boldtheta$, shallow', filename + '_random_shallow')
    table.add(label, r'Random $\boldtheta$, deep', filename + '_random_deep')

    table.add(label, r'Morphing basis', filename + '_basis')
    table.add(label, r'Morphing basis, shallow', filename + '_basis_shallow')
    table.add(label, r'Morphing basis, deep', filename + '_basis_deep')

print(table.print())

   carl (param.) & Baseline & 0.4231\\
    & Baseline, shallow & 0.1477\\
    & Baseline, deep & 0.8262\\
    & Baseline, large batches & 0.0877\\
    & Baseline, small batches & 0.2961\\
    & Baseline, const.\ LR & 0.6266\\
    & Baseline, const.\ LR, large batches & 0.4484\\
    & Baseline, const.\ LR, small batches & 0.4484\\
    & Baseline, small LR & 0.0459\\
    & Baseline, small LR, large batches & 0.0473\\
    & Baseline, small LR, small batches & \emph{0.0379}\\
    & Baseline, small const.\ LR & 0.0574\\
    & Baseline, small const.\ LR, large batches & 0.0445\\
    & Baseline, small const.\ LR, small batches & 0.0661\\
    & Baseline, large LR & 8.6159\\
    & Baseline, large LR, large batches & 2.7722\\
    & Baseline, large LR, small batches & 4.9290\\
    & Baseline, large const.\ LR & 7.2075\\
    & Baseline, large const.\ LR, large batches & 6.2696\\
    & Baseline, large const.\ LR, small batches & 7.2000\\
    & Random $\boldtheta$ & 0.3858\\
    & Random $\boldtheta